In [1]:
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np

from citipy import citipy

In [2]:
# generating the random latitude and longitude arrays
# storing them in variables
# packing the latitude and longitude values as pairs using the zip function

lats = np.random.uniform(-90.000, 90.000, size=1500)

lngs = np.random.uniform(-180.000, 180.000, size=1500)

lats_lngs = zip(lats, lngs)

lats_lngs

# The output of this cell will be a zip object in memory

# Notes on above code block

- The zip function packs each pair of lats and lngs with the same index into a tuple
    - Here, that produces 1500 tuples of paired latitudes and longitues 
        - each latitude and longitude in a tuple can be accessed by the index of 0 and 1 respectively
    - cf the jupyter notebook API_Practice.ipynb for practice packing and unpacking zipped tuples

In [3]:
# adding the latitude and longitudes to a list
# means we only need to create the random sets once
# NOTE you can only unzup a zipped tuple once before it's removed from computer's memory.

coordinates = list(lats_lngs)

In [5]:
# creating an empty list to hold the cities

cities = []

# identifying nearest city for each latitude and longitude combination

for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city name is unique, add to the cities []
    # The for-loop could find duplicate cities, and we only want unique cities
    
    if city not in cities:
        cities.append(city)
        
# print the city count to confirm sufficient count:
# Note that citipy finds the nearest city to the latitude and longitude pair with a population of 500+

print(len(cities))

622
